In [1]:
import sqlite3
import time
import pandas as pd
import numpy as np
import query
import preprocesser
import datetime


def non_decreasing(L):
    data = [x<=y for x, y in zip(L, L[1:])]
    return data

In [2]:
#1.종목명
shcode_list = query.query_shcode_list_today()

#2.target from csv
df = pd.read_csv("high_ratio_0.15_volume_money_30000000000_v2.csv", dtype ={'shcode':str})

#3. target index 추출
fin_result = pd.DataFrame([],columns = ['target_day','shcode'])

for i,v in df['index'].items():
    shcode = df['shcode'].iloc[i].zfill(6)
    target_date = v.split(" ")
    target_day_list = pd.to_datetime(target_date, format = "%Y-%m-%d")
    interval_end_timestamp = target_day_list + datetime.timedelta(weeks=6)
    result = pd.DataFrame(dict(target_day = target_day_list.strftime("%Y%m%d"),
                              interval_end = interval_end_timestamp.strftime("%Y%m%d"))).reset_index()
    del result['index']
    result['shcode'] = shcode
    fin_result = fin_result.append(result)
#4. make target month
fin_result['target_m'] = fin_result['target_day'].str.slice(stop = 6)
fin_result = pd.merge(fin_result, shcode_list, how = 'inner', on = 'shcode')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [12]:
'''
기간한정(8월대상한정)
#fin_result.groupby(['target_m']).count()
target_month = '201907'
fin_result = fin_result[fin_result['target_m'] == target_month]
fin_result = fin_result.reset_index()
#target_fin_result[['target_m','shcode','shname','target_day']].to_csv("201907.csv",encoding = "ms949")
'''

In [3]:
fin_result.head()

,interval_end,shcode,target_day,target_m,shname
0,20190909,000140,20190729,201907,하이트진로홀딩스
1,20190626,000890,20190515,201905,보해양조
2,20190424,001140,20190313,201903,카리스국보
3,20191029,001340,20190917,201909,백광산업
4,20190619,001550,20190508,201905,조비


In [ ]:
'''
#1. 이평 정배열 확인한다.

# 대세 판단1 : 월봉 20이평이 증가 중 12개?
# 대세 판단2 : 월봉 5이평이 증가 중
# 대세 판단3 : 월봉 ma5 - ma20  크눈중

#대세 판단 비고 : 외인 비율 확인하기
# 월봉,주봉 이평정배열(이평 5가 20보다 위인 any) or 
# 눌림목 발생하고 ,  거래량 똥 망하고,
# range 사이의 어디구간 30%~50%(구분지어서볼것), 
'''
'''
#2. 시점기준 관련된 주체 확인하여 쿼리로 작성한다. 최근관련된정보 한해서 확인(7,8 등)

'''

In [ ]:
'''
# 눌림목 발생하고 
sqeezed_bool = abs(df['close'] /df['open'] - 1) <= 0.03
#거래량 3/100 망하고,
volume_bool = df['volume'] / df['volume'].iloc[0] <= 0.03
# range 사이의 범위
spot_range_value = (df['close'] - df['open'].iloc[0]) / (df['close'].iloc[0] - df['open'].iloc[0])
# inrange 여부
inrange_bool = (df['close'] >= df['open'].iloc[0])&(df['close'] <= df['close'].iloc[0])

test_list = pd.DataFrame(dict(date = np.array(df)[:,0][1:],
                              close_ratio = [y/x-1 for x, y in zip(np.array(df)[:,4][:-1], np.array(df)[:,4][1:])],
                              close = df['close'][1:],
                              spot_range = spot_range_value[1:],
                              check = sqeezed_bool[1:] * volume_bool[1:] * inrange_bool[1:],
                              squeezed = sqeezed_bool[1:],
                              volume_default = volume_bool[1:],
                              inrange = inrange_bool[1:]
                             ))


'''
'''
# daily open <= open
open_touch_list = df['date'].iloc[1:][df['open'] <= open_trigger]
# daily close >= close
close_touch_list = df['date'].iloc[1:][df['close'] >= close_trigger]

close_touch_list
'''

In [ ]:
amount = [0.15, 0.15, 0.2, 0.2, 0.3]
# 대세 판단1 : 월봉 5이평이 감소하지않음
ma5 = non_decreasing(df['close_ma5'])[-5:]
ma5_score = sum(x * y for x, y in zip(ma5, amount)) / sum(amount)
# 대세 판단2 : 월봉 20이평이 감소하지않음
ma20 = non_decreasing(df['close_ma20'])[-5:]
ma20_score = sum(x * y for x, y in zip(ma20, amount)) / sum(amount)
# 대세 판단3 : 월봉 ma5 - ma20  크눈중
ma5_20 = non_decreasing(df['close_ma5']-df['close_ma20'])[-5:]
ma5_20_score = sum(x * y for x, y in zip(ma5_20, amount)) / sum(amount)

In [4]:
fin_result.head()

,interval_end,shcode,target_day,target_m,shname
0,20190909,000140,20190729,201907,하이트진로홀딩스
1,20190626,000890,20190515,201905,보해양조
2,20190424,001140,20190313,201903,카리스국보
3,20191029,001340,20190917,201909,백광산업
4,20190619,001550,20190508,201905,조비


In [7]:
fin_result.iloc[384:]

,interval_end,shcode,target_day,target_m,shname
384,20190227,198440,20190130,201901,고려시멘트
385,20190705,198440,20190607,201906,고려시멘트
386,20190321,203650,20190221,201902,드림시큐리티
387,20190716,203650,20190618,201906,드림시큐리티
388,20190508,205100,20190410,201904,엑셈
389,20190208,206560,20190111,201901,덱스터
390,20190211,206560,20190114,201901,덱스터
391,20190514,206560,20190416,201904,덱스터
392,20190726,207760,20190628,201906,미스터블루
393,20190801,207760,20190704,201907,미스터블루


In [ ]:
# 001745 SK네트웍스우(i = 6) : rotate_ratio , i=6 회전율볼것
# 002070 남영비비안 i = 8 : 거래대금 이하의 range 범위 변경

In [5]:
target_day_list = pd.to_datetime(fin_result['target_day'], format = "%Y-%m-%d")
check_init_timestamp = target_day_list - datetime.timedelta(days=60)
fin_result['check_init_day'] = check_init_timestamp.apply(lambda x: x.strftime("%Y%m%d"))

In [36]:
df = query.query_t1305(shcode = fin_result['shcode'].iloc[i], 
                      fr = fin_result['check_init_day'].iloc[i], 
                      to = fin_result['target_day'].iloc[i])

In [38]:
df.columns

Index(['date', 'open', 'high', 'low', 'close', 'volume', 'deal_f',
       'burn_ratio', 'rotate_ratio', 'for_net_buy', 'com_net_buy',
       'per_net_buy'],
      dtype='object')

In [42]:
i = 378
df = query.query_t1305(shcode = fin_result['shcode'].iloc[i], 
                      fr = fin_result['check_init_day'].iloc[i], 
                      to = fin_result['target_day'].iloc[i])

df['daily_ratio'] = df['close'] / df['close'].shift(1) - 1
df['volume_money'] = df['close'] * df['volume']

In [43]:
print(fin_result['shcode'].iloc[i] , fin_result['shname'].iloc[i])

179900 유티아이


In [45]:
df[['date','open','high','low','close','daily_ratio','volume','volume_money','burn_ratio', 'rotate_ratio']].tail(10)

,date,open,high,low,close,daily_ratio,volume,volume_money,burn_ratio,rotate_ratio
33,20190628,20100,20300,18500,19000,-0.052369,134361,2552859000,5.98,0.85
34,20190701,19000,19700,19000,19150,0.007895,59478,1139003700,6.03,0.37
35,20190702,19250,19800,18850,19700,0.028721,50399,992860300,5.99,0.32
36,20190703,19900,20000,18950,19200,-0.025381,46613,894969600,5.93,0.29
37,20190704,18950,19200,16650,17700,-0.078125,333998,5911764600,6.00,2.11
38,20190705,17700,18050,17250,18000,0.016949,83959,1511262000,5.99,0.53
39,20190708,18300,18300,16800,17300,-0.038889,151181,2615431300,6.06,0.95
40,20190709,17800,17850,17000,17500,0.011561,57191,1000842500,6.01,0.36
41,20190710,17500,18250,17300,18250,0.042857,56218,1025978500,6.03,0.35
42,20190711,18500,23700,18500,23700,0.298630,3377296,80041915200,5.73,21.29


In [ ]:
df['date'].tail(10)[df['daily_ratio'].tail(10)>= 0]

In [66]:
df.tail(10).iloc[::-1]

,date,open,high,low,close,volume,deal_f,burn_ratio,rotate_ratio,for_net_buy,com_net_buy,per_net_buy,daily_ratio,volume_money
42,20190711,18500,23700,18500,23700,3377296,132.22,5.73,21.29,-46760,14998,47092,0.298630,80041915200
41,20190710,17500,18250,17300,18250,56218,155.64,6.03,0.35,1632,0,-1632,0.042857,1025978500
40,20190709,17800,17850,17000,17500,57191,60.04,6.01,0.36,-7340,0,7308,0.011561,1000842500
39,20190708,18300,18300,16800,17300,151181,40.94,6.06,0.95,11705,-11578,-829,-0.038889,2615431300
38,20190705,17700,18050,17250,18000,83959,53.04,5.99,0.53,-1552,-19818,15684,0.016949,1511262000
37,20190704,18950,19200,16650,17700,333998,55.84,6.00,2.11,6819,-18465,9404,-0.078125,5911764600
36,20190703,19900,20000,18950,19200,46613,38.85,5.93,0.29,-8062,-1981,11069,-0.025381,894969600
35,20190702,19250,19800,18850,19700,50399,132.60,5.99,0.32,-5780,-800,6580,0.028721,992860300
34,20190701,19000,19700,19000,19150,59478,86.78,6.03,0.37,8136,0,-8701,0.007895,1139003700
33,20190628,20100,20300,18500,19000,134361,52.06,5.98,0.85,-2114,-5335,7239,-0.052369,2552859000


In [67]:
init_range = df['close'].iloc[-2]
for i,v in df['close'].iloc[::-1].iloc[1:].items():
    if v <= init_range:
        init_range = v
        continue
    else:
        break

In [68]:
init_range

17300

In [54]:
df['date'].tail(10)[df['daily_ratio'].tail(10)>= 0]

34    20190701
35    20190702
38    20190705
40    20190709
41    20190710
42    20190711
Name: date, dtype: object

In [3]:
rrr = pd.DataFrame([],columns = ['date',  'close_ratio',
                                 'min_target', 'max_target','volume_target', 'close','volume' , 'spot_range',
                                'check', 'inrange', 'squeezed', 'volume_default',
                                'shcode','shname', 'target_m', 'index',
                                 'deal_f','burn_ratio','rotate_ratio','for_net_buy','com_net_buy','per_net_buy'
                                ])

for i in range(len(fin_result['shcode'])):
    # 월봉 대세판단
    df_monthly = query.query_ohlcv_month(shcode = fin_result['shcode'].iloc[i], 
                                      to = str(int(fin_result['target_day'].iloc[i][:6]) + 1) + '01')
    
    #except1: data없는경우
    if df_monthly.shape[0] ==0:
        continue
    
    # ma preprocesser
    df_monthly = preprocesser.sma_mothly(df_monthly)
    
    ma5_20 = non_decreasing(df_monthly['close_ma5']-df_monthly['close_ma20'])[-5:]
               
    amount = [0.15, 0.15, 0.2, 0.2, 0.3]
    ma5_20_score = sum(x * y for x, y in zip(ma5_20, amount)) / sum(amount)
    
    #except2: 이평선제외하자
    if ma5_20_score == 0:
        continue
    
    #일봉 
    df = query.query_t1305(shcode = fin_result['shcode'].iloc[i], 
                          fr = fin_result['target_day'].iloc[i], 
                          to = fin_result['interval_end'].iloc[i])
    
    range_open = df['open'].iloc[0]
    range_close = df['close'].iloc[0]
    target_volume = df['volume'].iloc[0]
    
    #close 재조정
    for j,v in df['close'].iloc[1:].items():
        if v >= range_close:
            range_close = v
        else:
            break
               
    # 눌림목 발생하고     
    sqeezed_bool = abs(df['close'] /df['open'] - 1) <= 0.03
    #거래량 3/100 망하고,
    volume_bool = df['volume'] / target_volume <= 0.1
    # range 사이의 범위
    spot_range_value = (df['close'] - range_open) / (range_close - range_open)
    # inrange 여부
    inrange_bool = (df['close'] >= range_open)&(df['close'] <= range_close)

    test_list = pd.DataFrame(dict(date = np.array(df)[:,0][1:],
                                  close_ratio = [y/x-1 for x, y in zip(np.array(df)[:,4][:-1], np.array(df)[:,4][1:])],
                                  min_target = range_open,
                                  max_target = range_close,
                                  volume_target = target_volume,
                                  close = df['close'][1:],
                                  volume = df['volume'][1:],
                                  spot_range = spot_range_value[1:],
                                  check = sqeezed_bool[1:] * volume_bool[1:] * inrange_bool[1:],
                                  inrange = inrange_bool[1:],
                                  squeezed = sqeezed_bool[1:],                                  
                                  volume_default = volume_bool[1:],
                                  deal_f = df['deal_f'][1:],
                                  burn_ratio = df['burn_ratio'][1:],
                                  rotate_ratio = df['rotate_ratio'][1:],
                                  for_net_buy = df['for_net_buy'][1:],
                                  com_net_buy = df['com_net_buy'][1:],
                                  per_net_buy = df['per_net_buy'][1:]
                                 ))
    test_list['shcode'] = fin_result['shcode'].iloc[i]
    test_list['shname'] = fin_result['shname'].iloc[i]
    test_list['target_m'] = fin_result['target_m'].iloc[i]
    test_list['index'] = i
    rrr = rrr.append(test_list)
    print(i , fin_result['shcode'].iloc[i] , "완료")

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


0 000140 완료
1 000890 완료
2 001140 완료
3 001340 완료
6 001745 완료
8 002070 완료
9 002070 완료
10 002140 완료
11 002140 완료
12 002140 완료
13 002150 완료
16 002320 완료
17 002360 완료
18 002450 완료
19 002700 완료
20 002720 완료
21 002780 완료
22 002780 완료
23 002995 완료
24 003010 완료
25 003495 완료
27 003850 완료
28 004060 완료
29 004270 완료
30 004720 완료
31 004770 완료
32 004830 완료
34 004980 완료
35 005110 완료
36 005110 완료
37 005320 완료
38 005360 완료
39 005390 완료
42 005740 완료
43 006090 완료
44 006890 완료
45 006980 완료
46 006980 완료
47 006980 완료
48 007110 완료
49 007110 완료
50 007210 완료
52 007540 완료
53 007540 완료
54 008040 완료
55 008350 완료
57 008700 완료
58 008730 완료
59 008775 완료
60 009290 완료
61 009320 완료
62 009320 완료
64 009410 완료
65 009680 완료
66 009680 완료
67 010100 완료
68 010100 완료
69 010420 완료
70 010660 완료
71 011150 완료
72 011150 완료
73 011150 완료
74 011280 완료
75 011280 완료
76 011390 완료
77 012160 완료
78 012450 완료
79 012610 완료
80 012690 완료
81 012800 완료
82 012800 완료
83 013700 완료
84 014130 완료
85 014580 완료
86 014580 완료
87 014580 완료
88 014580 완료
89 014

In [4]:
rrr[['index','shcode', 'shname', 'target_m', 'date','close_ratio', 'min_target','max_target'
    ,'close','spot_range','check','inrange','squeezed','volume_default','volume_target','volume',
    'deal_f','burn_ratio','rotate_ratio','for_net_buy','com_net_buy','per_net_buy']].to_csv("3_3.csv",encoding = 'ms949')

In [16]:
i = 0
df_monthly = query.query_ohlcv_month(shcode = fin_result['shcode'].iloc[i], 
                                  to = str(int(fin_result['target_day'].iloc[i][:6]) + 1) + '01')


# ma preprocesser
df_monthly = preprocesser.sma_mothly(df_monthly)

ma5_20 = non_decreasing(df_monthly['close_ma5']-df_monthly['close_ma20'])[-5:]

amount = [0.15, 0.15, 0.2, 0.2, 0.3]
ma5_20_score = sum(x * y for x, y in zip(ma5_20, amount)) / sum(amount)

In [17]:
ma5_20_score

1.0

In [38]:
i = 2

In [39]:
#일봉 
df = query.query_ohlcv(shcode = fin_result['shcode'].iloc[i], 
                      fr = fin_result['target_day'].iloc[i], 
                      to = fin_result['interval_end'].iloc[i])

In [40]:
df

,date,open,high,low,close,volume
0,20190313,30150,30150,26200,27300,1303767
1,20190314,26550,26700,25000,25250,365876
2,20190315,26800,26950,22000,22850,487198
3,20190318,23200,29700,22700,29700,1060965
4,20190319,28650,28900,26550,26700,429929
5,20190320,26850,27000,22250,23200,502270
6,20190321,23550,25100,22700,23000,280727
7,20190322,23000,23050,21250,22250,218229
8,20190325,22050,22650,21100,21900,129333
9,20190326,22300,23200,21950,22900,125463


In [20]:
df['ratio'] = df['close'] / df['open'] - 1

In [22]:
range_open = df['open'].iloc[0]
range_close = df['close'].iloc[0]    

In [25]:
for j,v in df['close'].iloc[1:].items():
    if v >= range_close:
        range_close = v
    else:
        break

0 12000
1 12050
2 10650
3 9950
4 10400
5 12200
6 12350
7 11550
8 10950
9 10850
10 10950
11 10550
12 10550
13 10100
14 10150
15 10100
16 10300
17 10300
18 10600
19 10300


In [27]:
range_close

12050

In [24]:
for j in 
if df['close'].iloc[1:] >= range_close
    range_close

1      True
2     False
3     False
4     False
5      True
6      True
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
Name: close, dtype: bool

In [21]:
df.head()

,date,open,high,low,close,volume,ratio
0,20190729,9310,12000,9260,12000,6291621,0.288937
1,20190730,12350,14150,12000,12050,7616938,-0.024291
2,20190731,12150,12300,10450,10650,2326322,-0.123457
3,20190801,10650,11100,9820,9950,1521007,-0.065728
4,20190802,10400,10850,9970,10400,2206395,0.000000


#최종